## Scraping data about astronauts
*Author: Wes Bosse (SEA/LA/San Diego)*

First things first, bring in the requests library

In [2]:
import requests
import pandas as pd
import json

When we query an api using `requests.get()`, we will recieve a response back.  
Query `http://api.open-notify.org/iss-now.json` and save the response

In [3]:
response = requests.get('http://api.open-notify.org/iss-now.json')

#### Display the response's status code, keeping in mind the different status' we can recieve: 
- 200 - everything went okay, and the result has been returned (if any)
- 301 - the server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
- 401 - the server thinks you're not authenticated. This happens when you don't send the right credentials to access an API.
- 400 - the server thinks you made a bad request. This can happen when you don't send along the right data, among other things.
- 403 - the resource you're trying to access is forbidden -- you don't have the right permissions to see it.
- 404 - the resource you tried to access wasn't found on the server.
- 500 - the developer responsible for the resource you're trying to access is bad

In [5]:
response

<Response [200]>

#### What happens when we make a bad request? Request the endpoint `http://api.open-notify.org/iss-pass` and print the status code. 

In [6]:
response = requests.get('http://api.open-notify.org/iss-pass')
response

<Response [404]>

#### Using the docs at [OpenNotify](http://open-notify.org/Open-Notify-API/), can you figure out what is wrong? 'Fix' the  base request url and print the new status code.

In [8]:
response = requests.get('http://api.open-notify.org/iss-pass.json')
response.status_code

400

#### Still not 200? What else are we missing?

In [9]:
base_url = 'http://api.open-notify.org/iss-pass.json'
full_url = base_url + '?lat=38.9004&lon=77.0210'
response = requests.get(full_url)
response.status_code

200

#### Making the request with a query param dictionary instead of shoving the params into our url string

In [10]:
params = {
    'lat': 38.900,
    'lon': 77.0210
}
response = requests.get(base_url, params)
response.status_code

200

In [15]:
# Check out the url of the requst 
response.url

'http://api.open-notify.org/iss-pass.json?lat=38.9&lon=77.021'

#### Lets look at the data we are actually getting back using response.content

In [11]:
response.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1579800201, \n    "latitude": 38.9, \n    "longitude": 77.021, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 634, \n      "risetime": 1579803806\n    }, \n    {\n      "duration": 613, \n      "risetime": 1579809617\n    }, \n    {\n      "duration": 595, \n      "risetime": 1579863953\n    }, \n    {\n      "duration": 645, \n      "risetime": 1579869721\n    }, \n    {\n      "duration": 554, \n      "risetime": 1579875615\n    }\n  ]\n}\n'

#### And now to actually look at the data

In [13]:
print(response.content.decode())

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1579800201, 
    "latitude": 38.9, 
    "longitude": 77.021, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 634, 
      "risetime": 1579803806
    }, 
    {
      "duration": 613, 
      "risetime": 1579809617
    }, 
    {
      "duration": 595, 
      "risetime": 1579863953
    }, 
    {
      "duration": 645, 
      "risetime": 1579869721
    }, 
    {
      "duration": 554, 
      "risetime": 1579875615
    }
  ]
}



### A quick detour to talk about JSON

- JSON stands for JavaScript Object Notation
- JSON allows us to encode data structures like lists and dictionaries into strings and vice-versa
- JSON is our friend, and knowing it makes other developers hate us less. 

#### Lets import the JSON library and create a simple list with 3 items

#### Create a list and then dump it into a string using `json.dumps()`

#### Convert the string back into a list

### Now back to space
Make the same request as before, using Netanya, Israel's coords. `lat: 32.3215, lon:34.8532`

In [16]:
params = {
    'lat':32.3215,
    'lon':34.8532
}
response = requests.get(base_url, params)
response.status_code

200

In [19]:
# save the json as a dictionary object
data = response.json()
data

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1579800726,
  'latitude': 32.3215,
  'longitude': 34.8532,
  'passes': 5},
 'response': [{'duration': 215, 'risetime': 1579803481},
  {'duration': 358, 'risetime': 1579809330},
  {'duration': 599, 'risetime': 1579815083},
  {'duration': 625, 'risetime': 1579820878},
  {'duration': 377, 'risetime': 1579869402}]}

#### Let's take a look at the headers of the response. For example, what is the content type?

In [20]:
response.headers

{'Server': 'nginx/1.10.3', 'Date': 'Thu, 23 Jan 2020 17:32:56 GMT', 'Content-Type': 'application/json', 'Content-Length': '523', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

In [21]:
data['response']

[{'duration': 215, 'risetime': 1579803481},
 {'duration': 358, 'risetime': 1579809330},
 {'duration': 599, 'risetime': 1579815083},
 {'duration': 625, 'risetime': 1579820878},
 {'duration': 377, 'risetime': 1579869402}]

#### Lets create a new column using the 'risetime'

In [22]:
df = pd.DataFrame(data['response'])
df

,duration,risetime
0,215,1579803481
1,358,1579809330
2,599,1579815083
3,625,1579820878
4,377,1579869402


## Increase the number of risetimes

In [23]:
params = {
    'lat':32.3215,
    'lon':34.8532,
    'n':20 # add an n value for number of risetimes
}
response = requests.get(base_url, params)
response.status_code

200

In [25]:
big_data = response.json()

In [28]:
big_df = pd.DataFrame(big_data['response'])
big_df.head()

,duration,risetime
0,215,1579803481
1,358,1579809330
2,599,1579815083
3,625,1579820878
4,377,1579869402


#### Practice Time
Print out the number of people currently in space  
Create a dataframe where the rows represent a person in space, with two columns containing their name and the spacecraft that they are on